# Sentiment Analysis in the Browser

In this notebook, we will show how to create a `.air` file to perform image classification in the browser using a neural network.  To do this, we will utilize the CIFAR-10 dataset to build the initial model, prune the model using the `mann` package, and then package the model using the `aisquared` Python SDK.

## Dependencies

For this notebook, the following dependencies are required:

- `mann`
- `aisquared`

Both of these are available on [pypi](https://pypi.org) via `pip`.  The following cell also runs the commands to install these dependencies as well as imports them into the notebook environment, along with TensorFlow (which is a dependency of the `mann` package).

In [1]:
! pip install mann
! pip install aisquared

from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import aisquared
import mann

## Model Creation

Now that the required packages have been installed and imported, it is time to create the sentiment analysis model.  To do this, we have to first download and preprocess the data, create the model, prune the model so that it can perform well in the browser, and then package the model in the `.air` format.  The following cells will go through an in-depth explanation of each of the steps in this process.

In [2]:
# Loading the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = tf.image.resize(x_train, [30, 30])/255
x_test = tf.image.resize(x_test, [30, 30])/255

label_map = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-02-03 12:36:45.717537: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-03 12:36:45.717626: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Create the model

input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = mann.layers.MaskedConv2D(
    16,
    padding = 'same',
    activation = 'relu'
)(input_layer)
x = mann.layers.MaskedConv2D(
    16,
    padding = 'same',
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MaskedConv2D(
    32,
    padding = 'same',
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    32,
    padding = 'same',
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    2,
    strides = 1,
    padding = 'valid'
)(x)
x = tf.keras.layers.Flatten()(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 30, 3)]       0         
_________________________________________________________________
masked_conv2d (MaskedConv2D) (None, 30, 30, 16)        896       
_________________________________________________________________
masked_conv2d_1 (MaskedConv2 (None, 30, 30, 16)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 16)        0         
_________________________________________________________________
masked_conv2d_2 (MaskedConv2 (None, 29, 29, 32)        9280      
_________________________________________________________________
masked_conv2d_3 (MaskedConv2 (None, 29, 29, 32)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 32)        0     

In [4]:
# Perform initial sparsification
model = mann.utils.mask_model(
    model,
    20,
    x = x_train[:1000],
    y = y_train[:1000]
)

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

# Create a pruning callback that will increase pruning rate as performance improves
callback = mann.utils.ActiveSparsification(
    performance_cutoff = 0.60,
    starting_sparsification = 20,
    max_sparsification = 80,
    sparsification_rate = 5
)

# Train the model with the sparsification callback
model.fit(
    x_train,
    y_train,
    epochs = 1000,
    batch_size = 512,
    validation_split = 0.2,
    verbose = 2,
    callbacks = [callback]
)

# Now that the model has been trained, convert all model layers to built-in TensorFlow layers
model = mann.utils.remove_layer_masks(model)

2022-02-03 12:36:59.233250: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/1000


2022-02-03 12:36:59.582654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-03 12:37:24.116212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


79/79 - 27s - loss: 1.8868 - accuracy: 0.3106 - val_loss: 1.8318 - val_accuracy: 0.3612
Performance measure set to val_accuracy
Model performance has not reached pruning threshold for 1 epoch(s)
Epoch 2/1000
79/79 - 25s - loss: 1.4783 - accuracy: 0.4714 - val_loss: 1.5425 - val_accuracy: 0.4555
Model performance has not reached pruning threshold for 2 epoch(s)
Epoch 3/1000
79/79 - 25s - loss: 1.3010 - accuracy: 0.5361 - val_loss: 1.2676 - val_accuracy: 0.5380
Model performance has not reached pruning threshold for 3 epoch(s)
Epoch 4/1000
79/79 - 25s - loss: 1.1497 - accuracy: 0.5899 - val_loss: 1.2129 - val_accuracy: 0.5665
Model performance has not reached pruning threshold for 4 epoch(s)
Epoch 5/1000
79/79 - 24s - loss: 1.0432 - accuracy: 0.6308 - val_loss: 1.2092 - val_accuracy: 0.5760
Model performance has not reached pruning threshold for 5 epoch(s)
Epoch 6/1000
79/79 - 24s - loss: 0.9319 - accuracy: 0.6736 - val_loss: 1.1509 - val_accuracy: 0.5910
Model performance has not reache

In [5]:
# Check model performance
preds = model.predict(x_test).argmax(axis = 1)
print('Model Performance on Test Data:')
print('\n')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

# Save the model
model.save('ImageClassifier.h5')

2022-02-03 12:46:57.215432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Model Performance on Test Data:


[[687  12  83  23  30   8  16  12  84  45]
 [ 29 710  19  20   8   7  18   5  38 146]
 [ 67   5 599  59  80  73  64  28  14  11]
 [ 34   8 116 428  76 182  75  34  18  29]
 [ 29   5 144  67 533  60  68  73  12   9]
 [ 18   3  99 148  61 557  37  52  13  12]
 [  9   7  77  64  45  39 727  14   8  10]
 [ 22   4  52  45  82  79  13 688   2  13]
 [ 71  33  22  13  15  14   5   6 784  37]
 [ 46 117  18  27  15  21  13  20  31 692]]
              precision    recall  f1-score   support

           0       0.68      0.69      0.68      1000
           1       0.79      0.71      0.75      1000
           2       0.49      0.60      0.54      1000
           3       0.48      0.43      0.45      1000
           4       0.56      0.53      0.55      1000
           5       0.54      0.56      0.55      1000
           6       0.70      0.73      0.71      1000
           7       0.74      0.69      0.71      1000
           8       0.78      0.78      0.78     

## Package the Model

Now that the model has been created, we can package the model into a single `.air` file that enables integration into the browser.

To perform this packaging, we will be utilizing the `aisquared` package Document

In [6]:
aisquared.base.ImagePredictor(
    'ImageClassifier.h5',
    'cifar10Classifier',
    image_size = [30, 30],
    divide_value = 255,
    label_map = label_map,
    include_probability = True
).compile()